In [43]:
import pandas as pd 
from pull_covid_data import pull_covid_data

pull_covid_data()

def load_to_memory(filename):
    data = pd.read_csv(f'{filename}', header=0) 
    return data

time_series = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
dataframe = load_to_memory(time_series)

def use_melt(df):
    print('Using Melt Function')
    melt_df = pd.melt(df, id_vars=['Country/Region'], value_vars=list(df.columns[4:]))
    melt_df.columns = ['Country/Region', 'Date', 'Cases']
    melt_df['Date'] = pd.to_datetime(melt_df['Date'])
    melt_df.insert(3, 'Computed Cases', None)
    return melt_df

    
    
use_melt(dataframe)


Here is the status update of COVID-19: 

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean
Using Melt Function


,Country/Region,Date,Cases,Computed Cases
0,Afghanistan,2020-01-22,0,None
1,Albania,2020-01-22,0,None
2,Algeria,2020-01-22,0,None
3,Andorra,2020-01-22,0,None
4,Angola,2020-01-22,0,None
...,...,...,...,...
143443,Vietnam,2021-06-20,13258,None
143444,West Bank and Gaza,2021-06-20,312673,None
143445,Yemen,2021-06-20,6885,None
143446,Zambia,2021-06-20,129033,None
